<a href="https://colab.research.google.com/github/JuanArocaMIAR/03MIAR---Algoritmos-de-Optimizacion---2023/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Juan_Aroca_Perez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Juan Aroca Perez <br>
Url: https://github.com/JuanArocaMIAR/03MIAR---Algoritmos-de-Optimizacion---2023.git<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>


**Descripción del problema:**

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

### **Problema 1:** Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

In [ ]:
#Respuesta
import pandas as pd
from IPython.display import display
import numpy as np

from google.colab import files

# Subir el archivo CSV desde tu sistema local
uploaded = files.upload()

# Seleccionar el archivo
archivo_csv = list(uploaded.keys())[0]

# Cargar el dataframe desde el archivo CSV
df = pd.read_csv(archivo_csv, index_col=0)

# Visualizar el dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

Saving df.csv to df.csv


,Actor,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
Toma,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,NaN,Total
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,3
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,4
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,3
6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4
7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,3
9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3


 La idea para resolver este programa es el empleo del problema tipo visto en la asignatura de **Programación lineal entera** puesto que, para empezar, todas las variables son de tipo entero. Además, cumple con la premisa de que hay que buscar un máximo o mínimo de una función objetivo lineal sujeto a restricciones, que en este caso, dichas restricciones serán que los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben y no es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

El objetivo del problema será **minimizar el gasto** total de los actores de doblaje. Por otro lado, como algoritmo de búsqueda de la solución, **vamos a emplear ramificación y poda**, para ello, necesitamos una estructura de árbol. Los actores y las tomas ya están colocados en la tabla.

 Las primeras **preguntas** que debemos hacernos son: ¿Cómo diseñamos el árbol? ¿Qué mecanismo diseñamos para ramificar? ¿Cómo podamos?.

 Una posible manera de **diseñar el árbol** es mediante una lista de tuplas, donde cada tupla representa una asignación parcial. Cada elemento de la tupla corresponde a un actor, y el valor en ese elemento indica a qué toma ha sido asignado ese actor como se vio en teoría. En el código que proporcionaste, una tupla del árbol de soluciones tiene la forma (a1, a2, ..., an), donde a_i es el número de toma a la cual el actor i ha sido asignado.

In [ ]:
# Estructura del árbol (lista de tuplas):
arbol = [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

# Función para generar hijos de un nodo:
def generar_hijos(nodo):
    hijos = []
    actores_no_asignados = [i for i in range(10) if nodo[i] == 0]
    for actor in actores_no_asignados:
        for toma in range(1, 11):
            hijo = list(nodo)
            hijo[actor] = toma
            hijos.append(tuple(hijo))
    return hijos

# Ramificación:
nodo_actual = arbol[0]
hijos = generar_hijos(nodo_actual)
print(hijos)

[(1, 0, 0, 0, 0, 0, 0, 0, 0, 0), (2, 0, 0, 0, 0, 0, 0, 0, 0, 0), (3, 0, 0, 0, 0, 0, 0, 0, 0, 0), (4, 0, 0, 0, 0, 0, 0, 0, 0, 0), (5, 0, 0, 0, 0, 0, 0, 0, 0, 0), (6, 0, 0, 0, 0, 0, 0, 0, 0, 0), (7, 0, 0, 0, 0, 0, 0, 0, 0, 0), (8, 0, 0, 0, 0, 0, 0, 0, 0, 0), (9, 0, 0, 0, 0, 0, 0, 0, 0, 0), (10, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 2, 0, 0, 0, 0, 0, 0, 0, 0), (0, 3, 0, 0, 0, 0, 0, 0, 0, 0), (0, 4, 0, 0, 0, 0, 0, 0, 0, 0), (0, 5, 0, 0, 0, 0, 0, 0, 0, 0), (0, 6, 0, 0, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0, 0, 0), (0, 8, 0, 0, 0, 0, 0, 0, 0, 0), (0, 9, 0, 0, 0, 0, 0, 0, 0, 0), (0, 10, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 2, 0, 0, 0, 0, 0, 0, 0), (0, 0, 3, 0, 0, 0, 0, 0, 0, 0), (0, 0, 4, 0, 0, 0, 0, 0, 0, 0), (0, 0, 5, 0, 0, 0, 0, 0, 0, 0), (0, 0, 6, 0, 0, 0, 0, 0, 0, 0), (0, 0, 7, 0, 0, 0, 0, 0, 0, 0), (0, 0, 8, 0, 0, 0, 0, 0, 0, 0), (0, 0, 9, 0, 0, 0, 0, 0, 0, 0), (0, 0, 10, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 

Tenemos una lista de tuplas que representa nuestra **espacio de soluciones**. Debido al volumen de datos, no es posible considerar una sola tupla como hemos visto en teoría, sería ineficiente. Las asignaciones parciales resultan más eficientes en este caso, donde cada nodo es una asignación parcial, lo que es aconsejable si vamos a realizar la técnica de ramificación y poda.

De este modo, por ejemplo, si tenemos la tupla (1, 3, 0, 2, 4, 5, 7, 6, 8, 9), significa que:

El actor 0 ha sido asignado a la toma 1.
El actor 1 ha sido asignado a la toma 3.
El actor 2 ha sido asignado a la toma 0.
El actor 3 ha sido asignado a la toma 2.
Y así sucesivamente.

**¿Cómo construimos la función objetivo?** Dado que **el objetivo es minimizar el gasto total** por los servicios de los actores de doblaje, necesitamos asignar un costo a cada toma asignada. **La matriz de costos** nos ayudará a determinar este costo para cada asignación de toma por actor.

In [ ]:
import random

# Tenemos 10 actores y 30 tomas.
num_actores = 10
num_tomas = 30

# Calculamos el número de tomas que cada actor debe realizar acorde a las restricciones.
tomas_por_dia = 6

# Creamos una matriz de costos con unos y ceros. 0 no realiza la toma, 1 realiza la toma.
matriz_costos = [[0] * num_actores for _ in range(num_tomas)]

# Hay que asignar seis unos en cada toma.
for j in range(num_tomas):
    actores_asignados = 0
    while actores_asignados < tomas_por_dia:
        i = random.randint(0, num_actores - 1)
        if matriz_costos[j][i] == 0:
            matriz_costos[j][i] = 1
            actores_asignados += 1

# Imprimimos por pantalla la matriz de costos:
print("Matriz de Costos:")
for fila in matriz_costos:
    print(fila)

Matriz de Costos:
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 0, 1, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1, 1, 0, 0, 1, 0, 1, 1, 1, 0]
[1, 1, 1, 0, 0, 1, 0, 1, 1, 0]
[1, 0, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 1, 0, 0, 1, 1, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 1]
[1, 0, 1, 1, 0, 0, 1, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 1, 0, 1, 0]
[1, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
[1, 0, 0, 0, 1, 1, 0, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 0, 0, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 1, 0, 1]
[0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 0]
[0, 1, 0, 0, 1, 1, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
[1, 1, 0, 0, 1, 1, 0, 1, 1, 0]
[1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
[0, 0, 1, 0, 1, 1, 1, 1, 1, 0]
[1, 0, 1, 0, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]


 Vamos a proveernos de una función para estimar la **cota inferior para una solución parcial**. Tenemos que hacer una estimación, esto es la que nos va a servir para podar. Se trata de una estrategia para conducir al algoritmo a que sea más eficiente, o, a que dirija la búsqueda para encontrar la solución óptima. Para la cota inferior lo que **buscamos es el mínimo de esas columnas** para que nos sirva como función de cota inferior.

In [ ]:
import numpy as np

# Nuestro conjunto de datos original:
COSTES = np.array([
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
], dtype=int)

# Solución parcial: (1, 3, 0, 2, 4, 5, 7, 6, 8, 9) significa que el actor 0 ha sido asignado a la toma 1 y el actor 1 a la toma 3, al actor 3 la toma 0, y, así sucesivamente.
S_PARCIAL = np.array([1, 3, 0, 2, 4, 5, 7, 6, 8, 9])

def valor(S, COSTES):
    VALOR = 0
    for i in range(len(S)):
        VALOR += COSTES[S[i]][i]
    return VALOR

def CI(S, COSTES):
    VALOR = 0
    for i in range(len(S)):
        VALOR += COSTES[i][S[i]]

    for i in range(len(S), COSTES.shape[1]):
        VALOR += np.min(COSTES[:, i])

    return VALOR

print("Valor de la solución parcial:", valor(S_PARCIAL, COSTES))
print("Costo inferior:", CI(S_PARCIAL, COSTES))

Valor de la solución parcial: 2
Costo inferior: 2


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Dado que tenemos 10 actores y 30 tomas, tenemos un espacio de soluciones de 30**10 ya que cada actor puede ser asignado a cualquiera de las 30 tomas. El valor del espacio de soluciones es muy grande por lo que destaca la complejidad de explorar todo el espacio de soluciones y la necesidad de algoritmos como ramificación y poda para encontrar soluciones óptimas de manera eficiente.

El número total de soluciones tiene una expresión factorial. Esto se debe a que cada actor tiene 30 tomas entre las cuales puede ser asignado, y hay 10 actores en total, lo que supone un orden de complejidad O(N!), por lo que la técnica de ramificación y poda puede ser muy útil para reducir el número de búsqueda de soluciones.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Lo siguiente será aplicar la **técnica de ramificación y poda**. Hemos definido el conjunto de datos que representa el problema de asignación de los actores por tomas. Hemos definido una matriz de costo asociado a cada actor por cada toma.

A continuación vamos a definir las funciones auxiliares: valor, CI, crear_hijos y ramificación_y_poda. Donde la función **valor** calcula el valor de una solución parcial, **CI** calcula una cota inferior para la solución parcial, **crear_hijos** genera los hijos de un nodo para la asignación de actores por tomas, y, la función principal **ramificación_y_poda** ejecuta el algoritmo ramificación y poda.

En la ramificación y poda se van a generar nodos y ramificar el árbol de búsqueda a través de un bucle. En cada iteración se seleccionará el nodo más prometedor basándose de la cota inferior. Se generarán los jijos mediante la función "crear_hijos". Como es necesario también realizar una poda, se eliminarán los nodos cuya cota inferior es mayor o igual a la mejor solución encontrada hasta el momento.

In [ ]:
#Respuesta


In [ ]:
def crear_hijos(NODO, COSTES):
    HIJOS = []
    for i in range(len(COSTES)):
        if i not in NODO:
            HIJOS.append({'s': tuple(list(NODO) + [i])})
    return HIJOS

# Ejemplo de uso
nodo_actual = (0, 1, 3)
nuevos_hijos = crear_hijos(nodo_actual, COSTES)
print(nuevos_hijos)

[{'s': (0, 1, 3, 2)}, {'s': (0, 1, 3, 4)}, {'s': (0, 1, 3, 5)}, {'s': (0, 1, 3, 6)}, {'s': (0, 1, 3, 7)}, {'s': (0, 1, 3, 8)}, {'s': (0, 1, 3, 9)}, {'s': (0, 1, 3, 10)}, {'s': (0, 1, 3, 11)}, {'s': (0, 1, 3, 12)}, {'s': (0, 1, 3, 13)}, {'s': (0, 1, 3, 14)}, {'s': (0, 1, 3, 15)}, {'s': (0, 1, 3, 16)}, {'s': (0, 1, 3, 17)}, {'s': (0, 1, 3, 18)}, {'s': (0, 1, 3, 19)}, {'s': (0, 1, 3, 20)}, {'s': (0, 1, 3, 21)}, {'s': (0, 1, 3, 22)}, {'s': (0, 1, 3, 23)}, {'s': (0, 1, 3, 24)}, {'s': (0, 1, 3, 25)}, {'s': (0, 1, 3, 26)}, {'s': (0, 1, 3, 27)}, {'s': (0, 1, 3, 28)}, {'s': (0, 1, 3, 29)}]


Ahora lo siguiente sería: **Inicializamos la solución inicialmente con todo vacío**, y, nos vamos a ir guardando una **variable que llamamos NODOS**, los nodos que debemos explorar, los nodos vivos, con lo cual lo que hemos hecho es inicializar la solución.

In [ ]:
#  Creamos el algoritmo de ramificación y poda para asignación de actores a tomas:
def ramificacion_y_poda(COSTES):
    DIMENSION = len(COSTES[0])
    MEJOR_SOLUCION = tuple(i for i in range(DIMENSION))
    CotaSup = valor(MEJOR_SOLUCION, COSTES)

    NODOS = []
    NODOS.append({'s': (), 'ci': CI((), COSTES)})

    iteracion = 0

Realizamos un **bucle iterativo**, lo que hacemos es ramificar, exploramos, **buscamos el nodo prometedor**. Hemos creado una **variable HIJOS** que va a guardar los hijos, porque lo que vamos a hacer posteriormente, es podarlos. **A continuación solamente añadiremos a NODOS a aquellos nodos que hayan pasado el filtro de la poda**.

In [ ]:
# Creamos una función para crear hijos en el contexto de asignación de actores a tomas:
def crear_hijos(NODO, N):
    HIJOS = []
    for i in range(N):
        if i not in NODO:
            HIJOS.append({'s': NODO + (i,)})
    return HIJOS

# Ahora el algoritmo de ramificación y poda para asignación de actores a tomas:
def ramificacion_y_poda(COSTES_ACTORES):
    DIMENSION = len(COSTES_ACTORES[0])
    MEJOR_SOLUCION = tuple(i for i in range(DIMENSION))
    CotaSup = valor(MEJOR_SOLUCION, COSTES_ACTORES)

    NODOS = []
    NODOS.append({'s': (), 'ci': CI((), COSTES_ACTORES)})

    iteracion = 0

    while len(NODOS) > 0:
        iteracion += 1

        nodo_prometedor = min(NODOS, key=lambda x: x['ci'])['s']


        # Se generan los hijos:
        HIJOS = [{'s': x['s'], 'ci': CI(x['s'], COSTES_ACTORES)} for x in crear_hijos(nodo_prometedor, DIMENSION)]

        # Revisamos la cota superior y nos quedamos con la mejor solución si llegamos a una solución final:
        NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
        if len(NODO_FINAL) > 0:
            if NODO_FINAL[0]['ci'] < CotaSup:
                CotaSup = NODO_FINAL[0]['ci']
                MEJOR_SOLUCION = NODO_FINAL[0]['s']

        # Poda:
        HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]

        # Añadimos los hijos:
        NODOS.extend(HIJOS)

        # Eliminamos el nodo ramificado:
        NODOS = [x for x in NODOS if x['s'] != nodo_prometedor]

    print("La solución final es:", MEJOR_SOLUCION, "en", iteracion, "iteraciones", "para dimension:", DIMENSION)

# Usamos nuestra matriz de costos para actores.
COSTES_ACTORES = np.array([
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
], dtype=int)

ramificacion_y_poda(COSTES_ACTORES)

La solución final es: (5, 1, 0, 2, 4, 6, 7, 3, 8, 9) en 69690 iteraciones para dimension: 10
